# Solar Thau - Visualitzem la producció solar

### 1) Carreguem les dades i preparem algunes variables pels grafics

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import ipywidgets as widgets

# Load the dataset
df = pd.read_csv("data_solarthau_lite.csv")

# Compute net consumption
df["netConsumption"] = df["kWh"] - df["solar_kWh"]

# Dropdown for month selection
month_selector = widgets.Dropdown(
    options=[(f"Month {i}", i) for i in range(1, 13)], 
    value=1,
    description="Month:"
)

# kW range slider
kw_range = widgets.FloatSlider(value=200.0, min=0.0, max=200.0, step=10.0, description="Kw:")

# Price range slider
price_range = widgets.FloatSlider(value=5.0, min=0.0, max=300.0, step=0.05, description="Price for solar output:")

# Avoided emission range slider
avoided_emission_range = widgets.FloatSlider(value=0.35, min=0.0, max=1.0, step=0.05, description="Avoided Emission:")

In [2]:
df.head()

,month,day,hour,dow,kWh,solar_kWh,netConsumption
0,1,1,0,5,29,0.0,29.0
1,1,1,1,5,27,0.0,27.0
2,1,1,2,5,29,0.0,29.0
3,1,1,3,5,27,0.0,27.0
4,1,1,4,5,25,0.0,25.0


### 2) Dibuixem el consum per mes

In [3]:
# Compute hourly average profiles by month
df_avg = df.groupby(["month", "hour"]).mean().reset_index()

# Create a time column for visualization
df_avg["time"] = df_avg["hour"].astype(str) + ":00"

# Plot the data with one subplot per month
fig = px.line(df_avg, x="time", y=["kWh", "solar_kWh", "netConsumption"],
              facet_col="month", facet_col_wrap=4,
              labels={"value": "Energy (kWh)", "time": "Hour of Day"},
              title="Hourly Average Energy Profiles by Month")

# Show the interactive chart
fig.show()

Aqui els podem veure un per un.

In [4]:
def monthly_plot(month, kw=200):
    
    df_filtered = df[df["month"] == month]

    # Update kWh solar to scale with kw
    df_filtered.loc[:,["solar_kWh"]] = df_filtered["solar_kWh"] * kw / 200
    df_filtered.loc[:,["netConsumption"]] = df_filtered["kWh"] - df_filtered["solar_kWh"]
    
    # Create averages
    df_avg = df_filtered.groupby("hour").mean().reset_index()
    df_avg["time"] = df_avg["hour"].astype(str) + ":00"

    # Plot the data
    fig = px.line(df_avg, x="time", y=["kWh", "solar_kWh", "netConsumption"],
                  labels={"value": "Energy (kWh)", "time": "Hour of Day"},
                  title=f"Hourly Average Energy Profile - Month {month}");
    
    fig.show()

In [5]:
widgets.interactive(monthly_plot, month=month_selector, kw=kw_range)

interactive(children=(Dropdown(description='Month:', options=(('Month 1', 1), ('Month 2', 2), ('Month 3', 3), …

Tambe podem pensar en els patrons de consum de la setmana.

In [6]:
def weekly_plot(month, kw=200):
    df_filtered = df[df["month"] == month]
    # update kWh solar to scale with kw
    df_filtered.loc[:,["solar_kWh"]] = df_filtered["solar_kWh"] * kw / 200
    df_filtered.loc[:,["netConsumption"]] = df_filtered["kWh"] - df_filtered["solar_kWh"]
    df_avg_week = df_filtered.groupby(["hour","dow"]).mean().reset_index()
    
    # Create a time column that ensures correct ordering
    df_avg_week["time"] = df_avg_week["dow"] * 24 + df_avg_week["hour"]

    # Sort by day of the week and hour to maintain correct order
    df_avg_week = df_avg_week.sort_values(by=["month", "dow", "hour"])


    # Plot the data with one subplot per month
    fig = px.line(df_avg_week, x="time", y=["kWh", "solar_kWh", "netConsumption"],
                facet_col="month", facet_col_wrap=4,
                labels={"value": "Energy (kWh)", "time": "Day-Hour"},
                title="Weekly Hourly Average Energy Profiles by Month (168 Hours)")
    
    # Show the interactive chart
    fig.show()

In [9]:
widgets.interactive(weekly_plot, month=month_selector, kw=kw_range)

interactive(children=(Dropdown(description='Month:', options=(('Month 1', 1), ('Month 2', 2), ('Month 3', 3), …

### 3) Com calcular el benefici anual de la instal·lació solar fotovoltaica?

In [10]:
def economic_benefit(electricity_price_out = 5):
    # Defining the benefit as a function of the electricity price
    electricity_price_in = np.array([x for x in range(100, 400, 10)])
    benefit = [(sum(df[df["netConsumption"]>0]["solar_kWh"])*p + sum(df[df["netConsumption"]<=0]["kWh"])*p
               - sum(df[df["netConsumption"]<=0]["netConsumption"])*electricity_price_out)/1000.0 for p in electricity_price_in]
    
    # Plot the benefit as a function of cost per ton
    fig = px.line(x=electricity_price_in, y=benefit, labels={"x": "Electricity price (EUR/MWh)", "y": "Savings per year (EUR)"}, title="Economic Benefit of Solar Panels")
    
    # Show the interactive chart
    fig.show()

In [11]:
widgets.interactive(economic_benefit, electricity_price_out=price_range)

interactive(children=(FloatSlider(value=5.0, description='Price for solar output:', max=300.0, step=0.05), Out…

In [12]:
def factura_amb_i_sense(electricity_price_out = 5):
    # Defining the benefit as a function of the electricity price
    electricity_price_in = np.array([x for x in range(100, 400, 10)])
    cost_nosolar = [sum(df["kWh"])*p/1000.0 for p in electricity_price_in]
    cost_solar =  [(sum(df[df["netConsumption"]>0]["netConsumption"])*p +
               + sum(df[df["netConsumption"]<=0]["netConsumption"])*electricity_price_out)/1000.0 for p in electricity_price_in]
    
    # Plot the two costs with legend 
    fig = px.line(x=electricity_price_in, y=cost_nosolar, labels={"x": "Electricity price (EUR/MWh)", "y": "Cost per year (EUR)"}, title="Annual cost of electricity")
    fig.add_scatter(x=electricity_price_in, y=cost_solar, mode="lines", name="With solar panels", line=dict(dash="dash"))

    # Show the interactive chart
    fig.show()

In [13]:
widgets.interactive(factura_amb_i_sense, electricity_price_out=price_range)

interactive(children=(FloatSlider(value=5.0, description='Price for solar output:', max=300.0, step=0.05), Out…

### 4) Com calcular el benefici ambiental de la instal·lació solar fotovoltaica?

In [14]:
def environmental_benefit(kw=200, avoided_emissions=0.35):
    
    # Definining the benefit as a function of the cost per ton
    cost_per_ton = np.array([x * 1.0 for x in range(0, 200, 5)])
    benefit = sum(df["solar_kWh"]) * kw / 200.0 * avoided_emissions * cost_per_ton / 1000.0
    
    # Plot the benefit as a function of cost per ton
    fig = px.line(x=cost_per_ton, y=benefit, labels={"x": "Cost per Ton (EUR/MWh)", "y": "Benefit per year (EUR)"}, title="Environmental Benefit of Solar Panels")
    
    # Show the interactive chart
    fig.show()

In [15]:
widgets.interactive(environmental_benefit, kw=kw_range, avoided_emissions=avoided_emission_range)

interactive(children=(FloatSlider(value=200.0, description='Kw:', max=200.0, step=10.0), FloatSlider(value=0.3…